In [62]:
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array
from keras.callbacks import CSVLogger
from sklearn.metrics import classification_report

import modules.blocks as blocks
import modules.data as data
from modules.utilites import DataClass

In [63]:
# Model parametrs
class Input_pars(DataClass):
    seq_len = 100


class CN_pars(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_pars(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = False


class FF_pars(DataClass):
    dropout_rate = 0.1


class Optimaser_pars(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_pars(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_pars(DataClass):
    seq_len = 100
    cn = CN_pars()
    an = AN_pars()
    ff = FF_pars()
    optimazer = Optimaser_pars()
    training = Trainig_pars()


pars = Full_pars()

In [64]:
from python import LobFeatures

In [65]:
i= keras.Input(shape=(20,20))
x= keras.layers.Reshape((20,20,1))(i)

x= keras.layers.Conv2D(
    kernel_size=(1,2),
    filters=9,
    padding='same',
    # input_shape=(None, 20, 20),
)(x)

model = keras.Model(inputs=i, outputs=x)
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, 20, 20)]          0         
                                                                 
 reshape_25 (Reshape)        (None, 20, 20, 1)         0         
                                                                 
 conv2d_96 (Conv2D)          (None, 20, 20, 9)         27        
                                                                 
Total params: 27 (108.00 Byte)
Trainable params: 27 (108.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [66]:
# Model
inputs = blocks.input_block(pars.seq_len)
x= inputs


x = blocks.cnn_block(
    input_layer=x,
    filters=pars.cn.n_filters,
    dilation_steps=pars.cn.dilation_steps,
)

# x= keras.layers.Reshape(target_shape=(100,14,1))(x)
# x = LobFeatures.lob_inception(x)
# x= LobFeatures.lob_cnn(x)


x = blocks.norm_block(input_layer=x)

x = blocks.positional_encoder_block(input_layer=x)


x = blocks.transformer_block(
    input_layer=x,
    n_blocks=pars.an.blocks,
    n_heads=pars.an.attention_heads,
    share_weights=pars.an.share_weights,
)


x = blocks.ffn_block(
    input_layer=x,
    dropout_rate=pars.ff.dropout_rate,
)


model = keras.Model(inputs=inputs, outputs=x)
model.summary(
    line_length=100,
    expand_nested=True,
)

Model: "model_8"
____________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        
 input_38 (InputLayer)                       [(None, 100, 40)]                       0              
                                                                                                    
 conv1d_80 (Conv1D)                          (None, 100, 14)                         1134           
                                                                                                    
 conv1d_81 (Conv1D)                          (None, 100, 14)                         406            
                                                                                                    
 conv1d_82 (Conv1D)                          (None, 100, 14)                         406            
                                                                          

In [67]:
((x_train, y_train), (x_val, y_val), (x_test, y_test)) =\
data.load_dataset(horizon=4)

In [87]:
with open ('x_train.npy','wb') as file:
    np.save(file,x_train)
with open ('y_train.npy','wb') as file:
    np.save(file,y_train)

In [88]:
with open ('x_train.npy','rb') as file:
    x_train= np.load(file)
with open ('y_train.npy','rb') as file:
    y_train= np.load(file)

In [71]:
# Dataset
ds_train = data.build_dataset(
    x=x_train[:1000],
    y=y_train[:1000],
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_val = data.build_dataset(
    x=x_val[:1000],
    y=y_val[:1000],
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_test = data.build_dataset(
    x=x_test,
    y=y_test,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)
print(
    f'Train x: {str(x_train.shape): <15} - y: {y_train.shape}',
    f'Val   x: {str(x_val.shape): <15} - y: {y_val.shape}',
    sep='\n',
)

Train x: (203799, 40)    - y: (203799,)
Val   x: (50949, 40)     - y: (50949,)


In [72]:
# CompiLe
model.compile(
    keras.optimizers.Adam(
        learning_rate=pars.optimazer.lr,
        beta_1=pars.optimazer.adam_beta1,
        beta_2=pars.optimazer.adam_beta2,
        name="Adam",
    ),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.CategoricalAccuracy(name='accurancy'),
        keras.metrics.SparseCategoricalAccuracy(name='sparce_accurancy'),
    ],
)

In [74]:
# Train
model.fit(
    ds_train,
    epochs=pars.training.epochs,
    validation_data=ds_val,
    callbacks=[
        CSVLogger(r'logs/model1/log.csv', append=True, separator=';')
    ]
)

Epoch 1/150
2/2 [==============================] - 3s 2s/step - loss: 2.1650 - accurancy: 0.1088 - sparce_accurancy: 0.4273 - val_loss: 2.2386 - val_accurancy: 0.0089 - val_sparce_accurancy: 0.3829
Epoch 2/150
2/2 [==============================] - 3s 2s/step - loss: 2.1743 - accurancy: 0.2808 - sparce_accurancy: 0.4129 - val_loss: 2.2156 - val_accurancy: 0.0721 - val_sparce_accurancy: 0.3718
Epoch 3/150
2/2 [==============================] - 3s 2s/step - loss: 2.1533 - accurancy: 0.5516 - sparce_accurancy: 0.4107 - val_loss: 2.2086 - val_accurancy: 0.0821 - val_sparce_accurancy: 0.3707
Epoch 4/150
2/2 [==============================] - 3s 2s/step - loss: 2.1456 - accurancy: 0.4928 - sparce_accurancy: 0.4406 - val_loss: 2.2080 - val_accurancy: 0.0300 - val_sparce_accurancy: 0.3818
Epoch 5/150
2/2 [==============================] - 3s 2s/step - loss: 2.1195 - accurancy: 0.2597 - sparce_accurancy: 0.4673 - val_loss: 2.2022 - val_accurancy: 0.0155 - val_sparce_accurancy: 0.3818
Epoch 6/15

KeyboardInterrupt: 